<a href="https://colab.research.google.com/github/Jahid1990/Chatbot/blob/main/Step%20by%20step%20Chatbot%20building%20in%20telegram%20using%20python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ChatBoT

In this notebook, I will try to demostrate end to end  building process of telegram chatbot based in python. Chatbot has 3 parts if I breakdown components of a basic chatbot. 

1. Input
2. Understanding the Input
3. Provide answer based on  understanding the Input

I will use telegram as interface- to take input and show output, deep learning LSTM model to understand the Intent of the input.

lets explore......

# Required python libraries

In [ ]:
# !pip install bnlp_toolkit
import numpy as np # linear algebra
import pandas as pd 
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
pd.set_option("display.max_rows", 50000, "display.max_columns", 500)

# Training Data loading

I have around 88k queries and corresponding answers of that queries. I used this dataset for building this chatbot. Questions and answers would be limited to this dataset.*italicized text* 

In [ ]:
data=pd.read_csv("/content/drive/MyDrive/to_english.csv")
data=data.iloc[:,1:3]
data.columns=["Text","Intent"]
data.head()

,Text,Intent
0,Amar account e koto taka ache?,checkavailablebalanceand validity
1,Ami account balance kivabe dekhbo?,checkavailablebalanceand validity
2,Amar koto taka balance ase?,checkavailablebalanceand validity
3,Amar koto taka ache?,checkavailablebalanceand validity
4,Account er balance bolen,checkavailablebalanceand validity


# Text Processsing

Did not know that keras provide such wonderful functions for text processing. There are some few other libraries in pyhton which explored some of them but to me keras is awesome for its simplicity. I am happy using keras for text processing as well as training our intent understanding model.

following cells could be titled as follows:

1. Tokenizing, Sequencing and Padding
2. Train Test Splitting
3. Model Architecture
4. Compliling the mode along with loss function definition
5. Training model 

In [ ]:
max_fatures =2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ',char_level=True)
tokenizer.fit_on_texts(data['Text'].values)
#tokenizer.index_word=dict(pd.Series(tokenizer.index_word.values())[~pd.Series(tokenizer.index_word.values()).isin(stop_words)])
X = tokenizer.texts_to_sequences(data['Text'].values)
X = pad_sequences(X,padding='post')

In [ ]:
Y = pd.get_dummies(data['Intent'],prefix_sep='a')
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(59385, 102) (59385, 33)
(29250, 102) (29250, 33)


In [ ]:
embed_dim = 196
lstm_out = 200

model = Sequential()
model.add(Embedding(max_fatures, embed_dim, input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, return_sequences=True))
# model.add(LSTM(lstm_out, dropout=0.2, return_sequences=True))
# model.add(LSTM(lstm_out, dropout=0.2))
model.add(tf.keras.layers.Flatten())
model.add(Dense(Y_train.shape[1],activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 102, 196)          392000    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 102, 196)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 102, 200)          317600    
_________________________________________________________________
flatten_2 (Flatten)          (None, 20400)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 33)                673233    
Total params: 1,382,833
Trainable params: 1,382,833
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
import tensorflow as tf
learn_rate = 0.0001
optim = tf.keras.optimizers.Adam(lr=learn_rate, decay=1e-5)
checkpointer = ModelCheckpoint(filepath='/content/drive/MyDrive/gp_chatbot.h5', verbose=1, save_best_only=True)
model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])

In [ ]:
batch_size = 512
learn_rate = 0.0001
optim = tf.keras.optimizers.Adam(lr=learn_rate, decay=1e-5)
model.fit(X_train, Y_train, epochs = 100, batch_size=batch_size,validation_split=.2, verbose = 1)

Epoch 1/100
93/93 [==============================] - 7s 78ms/step - loss: 0.0206 - accuracy: 0.9932 - val_loss: 0.0823 - val_accuracy: 0.9779
Epoch 2/100
93/93 [==============================] - 5s 57ms/step - loss: 0.0198 - accuracy: 0.9931 - val_loss: 0.0772 - val_accuracy: 0.9798
Epoch 3/100
93/93 [==============================] - 5s 57ms/step - loss: 0.0204 - accuracy: 0.9930 - val_loss: 0.0771 - val_accuracy: 0.9784
Epoch 4/100
93/93 [==============================] - 5s 57ms/step - loss: 0.0196 - accuracy: 0.9934 - val_loss: 0.0762 - val_accuracy: 0.9789
Epoch 5/100
93/93 [==============================] - 5s 56ms/step - loss: 0.0192 - accuracy: 0.9936 - val_loss: 0.0790 - val_accuracy: 0.9788
Epoch 6/100
93/93 [==============================] - 5s 57ms/step - loss: 0.0186 - accuracy: 0.9937 - val_loss: 0.0766 - val_accuracy: 0.9796
Epoch 7/100
93/93 [==============================] - 5s 57ms/step - loss: 0.0184 - accuracy: 0.9940 - val_loss: 0.0773 - val_accuracy: 0.9783
Epoch 

#Evaluation

In [ ]:
from sklearn.metrics import classification_report
import tensorflow as tf
model=tf.keras.models.load_model('/content/drive/MyDrive/gp_chatbot.h5')
pred=model.predict(X_test)                                                                                                                                                                          
pred=np.argmax(pred,axis=1)
act=np.argmax(Y_test.to_numpy(),axis=1)
print(classification_report(pred, act))

In [ ]:
data["PRED"]=Y_test.columns[pd.DataFrame(pred).idxmax(axis=1)]
data["PRED_ACC"]=np.where(data.Intent==data.PRED,1,0)
data["PRED_ACC"].value_counts()/data.shape[0]*100
data

,Text,Intent,PRED,PRED_ACC
0,Amar account e koto taka ache?,checkavailablebalanceand validity,CheckSMSofferinfo,0
1,Ami account balance kivabe dekhbo?,checkavailablebalanceand validity,CheckSMSofferinfo,0
2,Amar koto taka balance ase?,checkavailablebalanceand validity,CheckSMSofferinfo,0
3,Amar koto taka ache?,checkavailablebalanceand validity,CheckSMSofferinfo,0
4,Account er balance bolen,checkavailablebalanceand validity,CheckSMSofferinfo,0
...,...,...,...,...
88630,What special minit offer there in my number,MicroVoiceOfferInfo,CheckSMSofferinfo,0
88631,Any minit offer are in my number,MicroVoiceOfferInfo,CheckSMSofferinfo,0
88632,Inform me about my minit offer in this number,MicroVoiceOfferInfo,CheckSMSofferinfo,0
88633,Which menit offer are available in my number,MicroVoiceOfferInfo,CheckSMSofferinfo,0


In [ ]:
data.PRED_ACC.value_counts()/data.shape[0]*100

0    99.299374
1     0.700626
Name: PRED_ACC, dtype: float64

In [ ]:
#!pip install google_trans_new
# from google_trans_new import google_translator  
# translator = google_translator()  
twt='Balns kivabe deke'
#twt=translator.translate(twt,lang_tgt='en')
twt=[twt]

#vectorizing
twt = tokenizer.texts_to_sequences(twt)
# #padding
twt = pad_sequences(twt,padding='post',maxlen=23,value=0.0,dtype='int32')
sentiment = model.predict(twt)
print(Y_test.columns[pd.DataFrame(sentiment).idxmax(axis=1)][0]+":"+" "+str(np.max(sentiment)))

checkavailablebalanceand validity: 0.99947625


In [ ]:
twt = 'internet kivabe kinte hoy'
twt=translator.translate(twt,lang_tgt='en')
twt

'How to buy internet '

In [ ]:
data

,Intent,Text,PRED
0,checkavailablebalanceand validity,How much money is in my account?,checkavailablebalanceand validity
1,checkavailablebalanceand validity,How do I view the account balance?,checkavailablebalanceand validity
2,checkavailablebalanceand validity,How much money do I have in balance?,checkavailablebalanceand validity
3,checkavailablebalanceand validity,How much money do I have?,checkavailablebalanceand validity
4,checkavailablebalanceand validity,Account is the balance ball,checkavailablebalanceand validity
...,...,...,...
88630,MicroVoiceOfferInfo,What special minit offer there in my number,MicroVoiceOfferInfo
88631,MicroVoiceOfferInfo,Any minute offer are in my number,MicroVoiceOfferInfo
88632,MicroVoiceOfferInfo,Inform me about my minit offer in this number,MicroVoiceOfferInfo
88633,MicroVoiceOfferInfo,Which menit offer are available in my number,MicroVoiceOfferInfo


In [ ]:
# text=" 10 ".join(data.iloc[:,0])
# #!pip install google_trans_new
# l=4500
# from google_trans_new import google_translator  
# translator = google_translator() 
# t=[]
# for i in range(20):
#   t.append(translator.translate(text[(i*l):(i+1)*l],lang_tgt='en'))

In [ ]:
!pip install punkt

ERROR: Could not find a version that satisfies the requirement punkt (from versions: none)
ERROR: No matching distribution found for punkt


In [ ]:
translator.translate('আমি বাংলায় গান গাই',lang_tgt='en')

'I sing in Bengali '

# Convert all text to English

In [ ]:
#!pip install google_trans_new

from google_trans_new import google_translator  
translator = google_translator()  
trans=[]
l=0
for i in data.iloc[:,0]:
  trans.append(translator.translate(i,lang_tgt='en')) 
  l=l+1 
  print(l)

In [ ]:
pd.concat([data,pd.DataFrame(trans)],axis=1).to_excel("/content/drive/MyDrive/to_english.xlsx")

In [ ]:
pd.DataFrame(trans).to_excel("/content/drive/MyDrive/to_english.xlsx")

# Tokenizer

In [ ]:
from bnlp import BasicTokenizer
basic_t = BasicTokenizer()
raw_text = "আমি বাংলায় গান গাই।"
tokens = basic_t.tokenize(raw_text)
print(tokens)

# output: ["আমি", "বাংলায়", "গান", "গাই", "।"]

punkt not found. downloading...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
['আমি', 'বাংলায়', 'গান', 'গাই', '।']


In [ ]:
from bnlp import NLTKTokenizer

bnltk = NLTKTokenizer()

text = "আমি ভাত খাই। সে বাজারে যায়। তিনি কি সত্যিই ভালো মানুষ?"

word_tokens = bnltk.word_tokenize(text)
sentence_tokens = bnltk.sentence_tokenize(text)
print(word_tokens)
print(sentence_tokens)

['আমি', 'ভাত', 'খাই', '।', 'সে', 'বাজারে', 'যায়', '।', 'তিনি', 'কি', 'সত্যিই', 'ভালো', 'মানুষ', '?']
['আমি ভাত খাই।', 'সে বাজারে যায়।', 'তিনি কি সত্যিই ভালো মানুষ?']


# STOP words

In [ ]:
from bnlp.corpus import stopwords,punctuations
from bnlp.corpus.util import remove_stopwords

# remove stop words
stopwords = stopwords()
raw_text = 'আমি ভাত খাই।'
result = remove_stopwords(raw_text, stopwords)
print(result)
# remove punctuations
final_result = remove_stopwords(raw_text, punctuations)
print(final_result)

['ভাত', 'খাই', '।']
['আমি', 'ভাত', 'খাই']


In [ ]:
from bnlp import BengaliGlove

bng = BengaliGlove()
glove_path = "/content/drive/MyDrive/bn_glove.39M.300d.txt"
word = "গ্রাম"
# res = bng.closest_word(glove_path, word)
# print(res)
vec = bng.word2vec(glove_path, word)


300

In [ ]:
!pip install langdetect

     |████████████████████████████████| 983kB 9.3MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993193 sha256=b52e9a2a28d218b9dbbf56718e7f48836b2ba16028e14ffdf22b3b1d097533d9
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d40a60ea06e6d669e8e5012e6c8b947a57
Successfully built langdetect


In [ ]:
#!pip install speechrecognition pyttsx3 googletrans
#!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
!pip install PyAudio
import speech_recognition as sr
r = sr.Recognizer()
#mic = sr.Microphone(device_index=0)

In [ ]:
#!pip install google_trans_new
from google_trans_new import google_translator  
translator = google_translator()  
twt='A variant (strain) of the coronavirus first identified in the United Kingdom is being seen in various countries around the world. This information was given in the report of BBC online on Sunday. This new type of corona has been identified in various European countries as well as Canada and Japan. The Corona strain has spread to Denmark, the Netherlands, Italy, Germany, Spain, Belgium, Sweden, Switzerland and France in Europe. Corona new strain has spread beyond Europe. This list includes Australia, South Africa, Lebanon, Singapore, Thailand. This strain was eventually identified in Japan and Canada. Japan first reported that at least five people had been diagnosed with new types of corona. They all recently came to the country from the UK. It was later reported that two more corona were identified in the country. One of them has been infected locally.'
twt=translator.translate(twt,lang_tgt='bn')
twt

'যুক্তরাজ্যে প্রথম চিহ্নিত করোনভাইরাসটির একটি বৈকল্পিক (স্ট্রেন) বিশ্বের বিভিন্ন দেশে দেখা যাচ্ছে। রবিবার বিবিসির অনলাইন প্রতিবেদনে এ তথ্য জানানো হয়েছে। এই নতুন ধরণের করোনাকে ইউরোপের বিভিন্ন দেশ পাশাপাশি কানাডা এবং জাপানে চিহ্নিত করা হয়েছে। করোনার স্ট্রেন ডেনমার্ক, নেদারল্যান্ডস, ইতালি, জার্মানি, স্পেন, বেলজিয়াম, সুইডেন, সুইজারল্যান্ড এবং ইউরোপের ফ্রান্সে ছড়িয়ে পড়েছে। করোনার নতুন স্ট্রেন ইউরোপ ছাড়িয়ে ছড়িয়ে পড়েছে। এই তালিকায় রয়েছে অস্ট্রেলিয়া, দক্ষিণ আফ্রিকা, লেবানন, সিঙ্গাপুর, থাইল্যান্ড। এই স্ট্রেনটি শেষ পর্যন্ত জাপান এবং কানাডায় চিহ্নিত হয়েছিল। জাপান প্রথম জানিয়েছিল যে কমপক্ষে পাঁচ জনকে নতুন ধরণের করোনায় ধরা পড়েছিল। তারা সকলেই সম্প্রতি যুক্তরাজ্য থেকে দেশে এসেছিলেন। পরে জানা গেছে যে দেশে আরও দুটি করোনাকে চিহ্নিত করা হয়েছিল। এর মধ্যে একটি স্থানীয়ভাবে সংক্রামিত হয়েছে। '